In [1]:
import requests
import re
from bs4 import BeautifulSoup

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"}
url = "https://dblp.org/"
search_url = "https://dblp.org/search/author"
proxy = "http://127.0.0.1:7890"

In [24]:
class DBLP(object):

    def __init__(self):
        self.session = requests.Session()
        # self.session.headers = headers
        self.proxy = None

    def set_proxy(self, proxy):
        if proxy:
            self.proxy = {
                "http": proxy,
                "https": proxy
            }
    
    def _get_soup(self, *args, **kwargs):
        if self.proxy:
            response = self.session.get(proxies=self.proxy, *args, **kwargs)
        else:
            response = self.session.get(*args, **kwargs)
        if response.status_code != 200:
            raise Exception("Failed to fetch webpage with status code " + str(response.status_code))
        return BeautifulSoup(response.content, "lxml")
    
    def retrieve_from_search(self, query):
        params = {"q": query}
        res = self._get_soup(search_url, params=params)
        author_list = res.find_all("a", href=re.compile("https://dblp.org/pid"))
        author = []
        if len(author_list):
            for entry in author_list:
                author.append((entry.get_text(), entry.get("href")))
            return author
        else:
            raise Exception("No matches")
    
    def select_author(self, author):
        num = len(author)
        print(str(num) + " author(s) found")
        for i, j in enumerate(author):
            print(str(i+1) + ": " + j[0] + ", link: " + j[1])
        select = int(input("Please select the author to download: "))
        if 1 <= select <= num:
            return author[select-1][1]
        else:
            raise Exception("Invalid value for selection")
    
    def download_list(self, select):
        response = self._get_soup(url=select)
        entry_list = response.find_all("cite", {"class": "data tts-content"})
        num = len(entry_list)
        paper_list = [0] * num
        for i, entry in enumerate(entry_list):
            text = entry.get_text()
            title = entry.find("span", {"class": "title"}).get_text()
            text = text.split(title, 1)
            authors = text[0][:-2]
            source = text[1][1:]
            paper_list[i] = [i+1, authors, title, source]
        return paper_list

In [25]:
dblp = DBLP()
dblp.set_proxy(proxy)
authors = dblp.retrieve_from_search("Ya-qin Zhang")
author = dblp.select_author(authors)
paper_list = dblp.download_list(author)
len(paper_list)

8 author(s) found
1: Ya-Qin Zhang, link: https://dblp.org/pid/09/2187
2: Ya-qin Zhang, link: https://dblp.org/pid/136/7647
3: Qin-Yan Zhang, link: https://dblp.org/pid/243/6931
4: Ya-Qin Zhang, link: https://dblp.org/pid/09/2187
5: Ya-qin Zhang, link: https://dblp.org/pid/136/7647
6: Yan-Qin Zhang, link: https://dblp.org/pid/310/6292
7: Yan-qin Zhang, link: https://dblp.org/pid/63/10187
8: Yanqing Zhang, link: https://dblp.org/pid/z/YanqingZhang


156

In [26]:
paper_list

[[1,
  'Xiaoxue Chen, Hao Zhao, Guyue Zhou, Ya-Qin Zhang',
  'CoRR abs/2109.05566 (2021)',
  'PQ-Transformer: Jointly Parsing 3D Objects and Layouts from Point Clouds.'],
 [2,
  'Xiaoxue Chen, Tianyu Liu, Hao Zhao, Guyue Zhou, Ya-Qin Zhang',
  'CoRR abs/2111.12608 (2021)',
  'Cerberus Transformer: Joint Semantic, Affordance and Attribute Parsing.'],
 [3,
  'Pengfei Li, Yongliang Shi, Tianyu Liu, Hao Zhao, Guyue Zhou, Ya-Qin Zhang',
  'CoRR abs/2111.14798 (2021)',
  'Semi-supervised Implicit Scene Completion from Sparse LiDAR.'],
 [4,
  'Xin Hu, Ya-Qin Zhang, Olivia W. Lee, Li Liu, Manshu Tang, Kent Lai, Matthew B. Boxer, Matthew D. Hall, Min Shen',
  'J. Comput. Aided Mol. Des. 33(4): 405-417 (2019)',
  'Discovery of novel inhibitors of human galactokinase by virtual screening.'],
 [5, 'Ya-Qin Zhang', 'VLSI-DAT 2012: 1', 'Advances in computing.'],
 [6,
  'Qian Zhang, Ya-Qin Zhang',
  'Proc. IEEE 96(1): 64-76 (2008)',
  'Cross-Layer Design for QoS Support in Multihop Wireless Networks.'